In [15]:
# Initial imports
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import date
from dateutil.relativedelta import relativedelta


In [4]:
# Load .env enviroment variables
load_dotenv()

True

In [22]:
# Set Universal variables:
selected_company="TSLA"
bench_mark="SPY"
n=5 #how many years of historical price to go by

today=date.today().strftime('%Y-%m-%d')
n_years_ago=(date.today() - relativedelta(years=n)).strftime('%Y-%m-%d')


In [5]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [25]:
# Format current date as ISO format
start_date = pd.Timestamp(n_years_ago, tz="America/New_York").isoformat()
end_date = pd.Timestamp(today, tz="America/New_York").isoformat()

# Set the tickers list to fetch info from Alpaca API

tickers = [selected_company, bench_mark]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df


In [29]:
# Reorganize the DataFrame
# Separate ticker data
# TSLA = df_ticker[df_ticker['symbol']=='TSLA'].drop('symbol', axis=1)
# SPY = df_ticker[df_ticker['symbol']=='SPY'].drop('symbol', axis=1)
df_ticker=df_ticker[['close','symbol']]
df_ticker['daily_return']=.pct_change()


,close,symbol
timestamp,,
2017-04-13 04:00:00+00:00,232.51,SPY
2017-04-17 04:00:00+00:00,234.57,SPY
2017-04-18 04:00:00+00:00,233.87,SPY
2017-04-19 04:00:00+00:00,233.44,SPY
2017-04-20 04:00:00+00:00,235.34,SPY
...,...,...
2022-04-07 04:00:00+00:00,1057.26,TSLA
2022-04-08 04:00:00+00:00,1025.49,TSLA
2022-04-11 04:00:00+00:00,975.93,TSLA


In [45]:
TSLA_close = TSLA[['close']]

In [47]:
TSLA_close

,close,daily_returns
timestamp,,
2015-12-01 05:00:00+00:00,237.19,NaN
2015-12-02 05:00:00+00:00,231.99,-0.021923
2015-12-03 05:00:00+00:00,232.71,0.003104
2015-12-04 05:00:00+00:00,230.38,-0.010012
2015-12-07 05:00:00+00:00,231.13,0.003255
...,...,...
2022-04-07 04:00:00+00:00,1057.26,0.010997
2022-04-08 04:00:00+00:00,1025.49,-0.030049
2022-04-11 04:00:00+00:00,975.93,-0.048328
